In [ ]:
# default_exp imgfeat

# imfeat

> Utilities for computing image features.

In [ ]:
# export

from skimage.feature import hog
import attr
from typing import Tuple
import numpy as np

@attr.s
class HogFactory(object):
    
    orientations=attr.ib(type=int, default=8)
    pixels_per_cell=attr.ib(type=Tuple[int,int], default=(8, 8))
    cells_per_block=attr.ib(type=Tuple[int,int], default=(1, 1))
    multichannel=attr.ib(type=bool, default=False)
    
    def __call__(self, arr: np.ndarray)->np.ndarray:
        return hog(arr,
               orientations=self.orientations,
               pixels_per_cell=self.pixels_per_cell,
               cells_per_block=self.cells_per_block,
               block_norm="L2",
               transform_sqrt=True,
               feature_vector=True,
               multichannel=self.multichannel
                  )

In [ ]:
im = np.random.randint(0, 255, size=(32,32))

print(HogFactory(orientations=8, pixels_per_cell=(8, 8),cells_per_block=(1, 1), multichannel=False)(im).shape)

(128,)


In [ ]:
# export

from skimage.feature import daisy
import attr
from typing import Tuple
import numpy as np

@attr.s
class DaisyFactory(object):
    
    step=attr.ib(type=int, default=8)
    radius=attr.ib(type=int, default=8)
    rings=attr.ib(type=int, default=2)
    histograms=attr.ib(type=int, default=6)
    orientations=attr.ib(type=int, default=8)
    
    def __call__(self, arr: np.ndarray)->np.ndarray:
        return daisy(arr, step=self.step, radius=self.radius, rings=self.rings, histograms=self.histograms,
                         orientations=self.orientations).flatten()

In [ ]:
print(DaisyFactory(orientations=8, step=8, radius=8, rings=2)(im).shape)

(416,)


/home/matthieu/anaconda2/envs/pypurr/lib/python3.7/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from int64 to float64
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
# export 

from numpy import ndarray

def flatten(arr: ndarray)->ndarray:
    return arr.flatten()

In [ ]:
# export

from skimage.feature import local_binary_pattern
import attr
from typing import Tuple
import numpy as np

@attr.s
class LBPFactory(object):
    
    radius=attr.ib(type=int, default=8)
    
    def __call__(self, arr: np.ndarray)->np.ndarray:
        return local_binary_pattern(arr, 8*self.radius, self.radius, "uniform").flatten()
    

In [ ]:
print(LBPFactory(radius=8)(im).shape)

(1024,)


In [ ]:
# export
import cv2
import numpy as np
import attr

from toolz import pipe
from toolz.curried import map as map_c
from itertools import chain

kernel = 0.2* np.random.randn(3,3,1024)

@attr.s
class ConvolutionFeature(object):
    kernel = attr.ib(type=np.ndarray)
    def __call__(self, arr: np.ndarray)->np.ndarray:
        return pipe(
                range(self.kernel.shape[-1]),
                map_c(lambda k: self.kernel[:,:,k]),
                map_c(lambda k: cv2.filter2D(arr, -1, k)),
                map_c(lambda out: np.concatenate([ np.array(cv2.meanStdDev(out)).flatten() , np.array([(out>0).mean(axis=(0,1))]) ], axis=0)),
                chain.from_iterable,
                list,
                np.array
            )

TODO: There seems to be a problem when kernel is over 512

In [ ]:
import numpy as np
import cv2
from skimage.data import chelsea
from skimage.color import rgb2gray
from matplotlib.pyplot import subplot, imshow, show, axis
import matplotlib.pyplot as plt

im = chelsea()
gray = rgb2gray(im)

kernel = 0.2* np.random.randn(3,3,1024)

out = ConvolutionFeature(kernel)(gray)

print(out.shape)

(3072,)
